In [ ]:
!pip install happytransformer

In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import DatasetDict, load_dataset, load_metric, ClassLabel, Sequence, Value
import pandas as pd
import torch
import string
import pickle

In [3]:
test_file = '/content/drive/MyDrive/Google Colab/Advanced NLP/Hannah_Datasets/miniC4.csv'
dataset = load_dataset('csv', data_files={'test':test_file})

Using custom data configuration default-b03359184f6619f3


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-b03359184f6619f3/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
model_path = '/content/drive/MyDrive/Google Colab/Advanced NLP/Syn_Model/'
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path, return_dict=True).to('cuda')

In [5]:
# takes about 30 mins time
def tokenize_function(sample):
    return tokenizer(sample['correct'], padding='max_length', truncation=True)
tokenized_datasets = dataset.map(tokenize_function, batched=True)
test_dataset = tokenized_datasets['test']

  0%|          | 0/5000 [00:00<?, ?ba/s]

In [22]:
# dictt = {}
# dictt['dataset'] = test_dataset

# C4_pickle = '/content/drive/MyDrive/Google Colab/Advanced NLP/Hannah_Datasets/C4_pickle.pickle'
# with open(C4_pickle, 'wb') as f:
#     pickle.dump(dictt, f)

In [4]:
# C4_pickle = '/content/drive/MyDrive/Google Colab/Advanced NLP/Hannah_Datasets/C4_pickle.pickle'
# test_dataset = None
# with open(C4_pickle, 'rb') as f:
#     test_dataset = pickle.load(f)['dataset']

In [ ]:
start_index = 301000 # start from 2500000
step = 16
C4_df = pd.DataFrame()
Parallel_C4 = '/content/drive/MyDrive/Google Colab/Advanced NLP/Hannah_Datasets/Parallel_C4.csv' 
cnt = 0

while True:
    if (start_index + step) > (len(test_dataset)-1):
        break
    end_index = start_index + step
    input_data = torch.tensor(test_dataset[start_index:end_index]['input_ids'])
    input_data = input_data.to('cuda')
    outputs = model.generate(input_data, max_length=256, min_length=1, do_sample=True, top_k=50)
    gen_samples = tokenizer.batch_decode(outputs)
    for i in range(len(gen_samples)):
        s = gen_samples[i]
        if "<unk>" in s:
            continue
        s=s.replace('<pad> ', "")
        s=s.replace('</s> ', "")
        s=s.replace('<pad>', "")
        incorrect = "grammar: " + s.translate(str.maketrans({key: " {0} ".format(key) for key in string.punctuation})) 
        correct = test_dataset[start_index+i]['correct'][42:]
        C4_df = C4_df.append({'input':incorrect, 'target':correct}, ignore_index=True)
    start_index += step
    cnt+=step
    if cnt>=1000:
      cnt=0
      C4_df.to_csv(Parallel_C4, mode='a', header=False, index=False)
      #C4_df.to_csv(Parallel_C4, index=False)
      C4_df = pd.DataFrame()
      print(start_index)

In [ ]:
Parallel_C4 = '/content/drive/MyDrive/Google Colab/Advanced NLP/Hannah_Datasets/Parallel_C4.csv'
tmp_df = pd.read_csv(Parallel_C4, index_col=None)
print(tmp_df.shape[0])